# Финальный ноутбук с инференсом.

## Подготовка данных.

### Импорт необходимых библиотек.

In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path

import torch
from torch.utils.data import DataLoader

from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
from catboost import CatBoostClassifier, Pool

### Загрузка данных.

In [2]:
# Путь к папке с датасетом
DATA_DIR = os.path.join('..', 'data')
TRAIN_DATA_DIR = os.path.join('..', 'data', 'train.csv')
TEST_DATA_DIR = os.path.join('..', 'data', 'test.csv')

# Путь к эмбеддингам текста.
TEXT_EMB_DIR = os.path.join('..', 'notebooks/text_emb')
IMG_EMB_DIR = os.path.join('..', 'notebooks/img_emb')

In [3]:
# Загрузка
df_train = pd.read_csv(TRAIN_DATA_DIR, index_col='id')
df_test = pd.read_csv(TEST_DATA_DIR, index_col='id')

### Базовая предобработка.

In [4]:
# Импортируем скрипт для работы с данными
import sys
sys.path.append(str(Path.cwd().parent))
from scripts import data_preprocess

**Нормализация**

In [5]:
# Нормализация.
train_processed, test_processed = data_preprocess.prepare_seller_features(df_train, df_test)

Общих продавцов: 1789
Новых продавцов в test: 1342


**Очистка**

In [6]:
# Очистка
import warnings
warnings.filterwarnings('ignore')

# Очистка и формаривание некоторых новых столбцов.
df_train_num, df_train_text = data_preprocess.clean_data(train_processed)
df_test_num, df_test_text = data_preprocess.clean_data(test_processed, type='test')

# Получим полные датасеты
df_train_full = df_train_num.merge(df_train_text.drop(columns=['ItemID', 'resolution']), how='left', on='id')
df_test_full = df_test_num.merge(df_test_text.drop(columns=['ItemID']), how='left', on='id')

**Загрузка эмбеддингов.**

In [7]:
# TEXT
embeddings, embeddings_test = data_preprocess.load_text_embeddings(dir=TEXT_EMB_DIR)
# IMAGES
train_img_emb, test_img_emb = data_preprocess.load_img_embeddings(dir=IMG_EMB_DIR)

### Получение датасетов.

**Тренировка (запуск необязателен)**

In [8]:
data_train_num, data_val_num, y_train, y_val, train_cat, val_cat, embeddings_train, embeddings_val, emb_img_train, emb_img_val, group_stats \
    = data_preprocess.df_extend(df_train_full, df_train_num, df_train_text, embeddings, 
                emb_img=train_img_emb, 
                df_type='train'
)

**Тест или инференс**

In [9]:
data_test_num, cat_data = data_preprocess.df_extend(df_test_full, df_test_num, df_test_text, embeddings_test, test_img_emb,
                                    df_type='test',
                                    group_stats=group_stats)

## Подготовка данных.

In [10]:
PCA = os.path.join('..', 'notebooks/pca')
pca_text, pca_image = data_preprocess.load_pcas(dir=PCA)

In [11]:
embeddings_test_p = pca_text.transform(embeddings_test)
emb_img_test_p = pca_image.transform(test_img_emb)

In [12]:
# Возьмем числовные данные
num_data_test = data_test_num.drop(columns='ItemID').values
# Теперь объединяем: эмбеддинги + числовые
X_test_num = np.concatenate([embeddings_test_p, emb_img_test_p, num_data_test], axis=1)  # (N, D + num_features)

In [15]:
# Объединяем в датафрейм для CatBoost

X_test = pd.concat([pd.DataFrame(X_test_num), cat_data.reset_index(drop=True)], axis=1)

cat_features_idx = list(range(X_test_num.shape[1], X_test_num.shape[1] + 2))

test_pool = Pool(X_test, cat_features=cat_features_idx)

## Загрузка модели.

In [17]:
model = CatBoostClassifier()
model.load_model('catboost_model.cbm')

# Инференс.

In [18]:
test_predictions = model.predict(test_pool)

submission = pd.DataFrame({
    'id': df_test.index,
    'prediction': test_predictions
})

submission.to_csv('submission.csv', index=False)


print(f"Создан файл submission.csv с {len(submission)} предсказаниями")
print(f"Распределение предсказаний:")
print(submission['prediction'].value_counts())
print()

Создан файл submission.csv с 31391 предсказаниями
Распределение предсказаний:
prediction
0    29586
1     1805
Name: count, dtype: int64

